# Creating Topography Variables

In [4]:
%load_ext jupyter_black
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import os
import geopandas as gpd
import rasterio

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [5]:
base_url = Path(os.getenv("STORM_DATA_DIR")) / "analysis/02_new_model_input/"
input_dir = base_url / "04_topography/input/srtm/"
output_dir = base_url / "04_topography/output/"

In [6]:
# reading in the files
os.listdir(input_dir)

# merging rasters

['srtm_59_08.zip',
 'srtm_59_09.zip',
 'srtm_59_10.zip',
 'srtm_59_11.zip',
 'srtm_60_08.zip',
 'srtm_60_09.zip',
 'srtm_60_10.zip',
 'srtm_60_11.zip',
 'srtm_61_08.zip',
 'srtm_61_09.zip',
 'srtm_61_10.zip',
 'srtm_61_11.zip',
 'srtm_62_08.zip',
 'srtm_62_10.zip',
 'srtm_62_11.zip',
 'srtm_62_09.zip']

## Slope

### Mean Slope

### Slope Deviation

## Mean Elevation

## Ruggedness

### Mean ruggedness

### Ruggedness deviation

## Area in Square Kilometers

## Coast

### With Coast

### Coast Length